In [2]:
import numpy as np
from scipy.optimize import minimize
import json
from sklearn.metrics import mean_squared_error
from torch.nn import GaussianNLLLoss
import torch

In [3]:
results_NN = {"v1":None,"v2":None}

#NN:
TRUTH_TRAIN_IDENTIFIER = "./multi_NN_{}/specie_{}_true_train_complete.npy"
TRUTH_TEST_IDENTIFIER = "./multi_NN_{}/specie_{}_true_test.npy"
FULL_TRAIN_IDENTIFIER = "./multi_NN_{}/specie_{}_model_no_{}_predictions_train_complete.npy"
FULL_TEST_IDENTIFIER = "./multi_NN_{}/specie_{}_model_no_{}_predictions_test.npy"
ONE_HOT_IDENTIFIER = "./multi_NN_{}/membership_specie_{}_model_{}_one_hot_train.npy"


"""
TRUTH_TRAIN_IDENTIFIER = "./multi_NN_{}/specie_{}_true_train_complete.npy"
TRUTH_TEST_IDENTIFIER = "./multi_NN_{}/specie_{}_true_test.npy"
FULL_TRAIN_IDENTIFIER = "./multi_kernel_{}/specie_{}_model_no_{}_full_test_pred.npy"
FULL_TEST_IDENTIFIER = "./multi_kernel_{}/specie_{}_model_no_{}_test_pred.npy"
ONE_HOT_IDENTIFIER = "./multi_kernel_{}/specie_{}_model_no_{}_one_hot.npy"
"""



for degree in ["v1","v2"]:
    train_true = {}
    test_true = {}
    pred_train = {}
    pred_test = {}
    one_hot = {}
    means_train = {}
    means_test = {}
    residuals_train = {}
    residuals_test = {}
   
    
    for specie in [1,6,7,8]:
        train_true[specie] = np.load(TRUTH_TRAIN_IDENTIFIER.format(degree,specie))
        test_true[specie] = np.load(TRUTH_TEST_IDENTIFIER.format(degree,specie))
        pred_train[specie] = []
        pred_test[specie] = []
        one_hot[specie] = []
        for model in range(16):
            pred_train[specie].append(np.load(FULL_TRAIN_IDENTIFIER.format(degree,specie,model)))
            pred_test[specie].append(np.load(FULL_TEST_IDENTIFIER.format(degree,specie,model)))
            one_hot[specie].append(np.load(ONE_HOT_IDENTIFIER.format(degree,specie,model)))
        
       
        pred_train[specie] = np.vstack(pred_train[specie]).T
        pred_test[specie] = np.vstack(pred_test[specie]).T
        one_hot[specie] = np.vstack(one_hot[specie]).T
        means_train[specie] = np.mean(pred_train[specie],axis=1)
        means_test[specie] = np.mean(pred_test[specie],axis=1)
        #residuals_train[specie] = np.abs(train_true[specie]- means[specie])
    
    results_NN[degree] = {"train_true":train_true,"test_true":test_true,"means_train":means_train,"means_test":means_test,"pred_train":pred_train,"pred_test":pred_test,"one_hot":one_hot}
    
    
    
ridge_results = {"v1":None,"v2":None}
ridge_results["v1"] = {"pred_train":{},"pred_test":{},"one_hot":{},"means_train":{},"means_test":{},"test_true":{k:results_NN["v1"]["test_true"][k] for k in [1,6,7,8]},"train_true":{k:results_NN["v1"]["train_true"][k] for k in [1,6,7,8]}}
ridge_results["v2"] = {"pred_train":{},"pred_test":{},"one_hot":{},"means_train":{},"means_test":{},"test_true":{k:results_NN["v2"]["test_true"][k] for k in [1,6,7,8]},"train_true":{k:results_NN["v2"]["train_true"][k] for k in [1,6,7,8]}}


for v in [1]:
    for species in [1,6,7,8]:
        pred_test = []
        pred_train = []
        one_hot_train = []
        
        for i in range(16):
                pred_test.append(np.load("./multi_ridge_v{}/specie_{}_model_no_{}_test_pred.npy".format(v,species,i)))
                pred_train.append(np.load("./multi_ridge_v{}/specie_{}_model_no_{}_full_test_pred.npy".format(v,species,i)))
                one_hot_train.append(np.load("./multi_ridge_v{}/specie_{}_model_no_{}_one_hot.npy".format(v,species,i)))
        
        pred_test = np.vstack(pred_test).T
        pred_train = np.vstack(pred_train).T
        one_hot_train = np.vstack(one_hot_train).T
    
        ridge_results["v1"]["pred_train"][species] = pred_train    
        ridge_results["v1"]["pred_test"][species] = pred_test
        ridge_results["v1"]["one_hot"][species] = one_hot_train
        ridge_results["v1"]["means_train"][species] = np.mean(pred_train,axis=1)
        ridge_results["v1"]["means_test"][species] = np.mean(pred_test,axis=1)               
        
for species, a_num in zip(["1H", "13C", "15N", "17O"],[1,6,7,8]):
    pred_test = np.load("./multi_ridge_v2/{}_test_commitee_prediction.npy".format(species))
    pred_train = np.load("./multi_ridge_v2/{}_commitee_prediction.npy".format(species))
    one_hot_train = np.load("./multi_ridge_v2/{}_train_binary_1_hot_N_sample_M_models.npy".format(species))
    
                       
    ridge_results["v2"]["pred_train"][a_num] = pred_train    
    ridge_results["v2"]["pred_test"][a_num] = pred_test
    ridge_results["v2"]["one_hot"][a_num]= one_hot_train
    ridge_results["v2"]["means_train"][a_num] = np.mean(pred_train,axis=1)
    ridge_results["v2"]["means_test"][a_num] = np.mean(pred_test,axis=1)

In [4]:
testing = True

In [13]:
result_dict = ridge_results

"""
def minimize(*args,**kwargs):
    mode = kwargs.pop("mode")
    if mode == "local":
        return scipy.optimize.minimize(*args,**kwargs)
    elif mode == "global":
        for_local_minimizer = kwargs.pop("args")
        return scipy.optimize.basinhopping(*args,**kwargs,niter=100,T=0.5,minimizer_kwargs={"args":for_local_minimizer})
    else:
        raise NotImplementedError
"""

def neglog_likelihood(ystd,ymeans,ytrue):
    #**2
    #**2
    ll = np.mean(-0.5*np.log(2*np.pi*(ystd**2)) - (ymeans-ytrue)**2/(2*(ystd**2)))
    return -ll

def neglog_likelihood_rescale(x,ystd,ymeans,ytrue,rescale,nll=neglog_likelihood):
    ystd = rescale(ystd,*x)
    return nll(ystd,ymeans,ytrue)

def rescale_a(ystd,alpha):
    return np.sqrt(alpha**2 * ystd**2)

def rescale_a_g(ystd,alpha,gamma):
    return np.sqrt(alpha**2 * ystd**(gamma+2))    

def rescale_a_b_g(ystd,alpha,beta,gamma):
    return np.sqrt(alpha**2 * ystd**(gamma+2) + beta**2)   

def neglog_likelihood_torch(ystd,ymeans,ytrue):
    loss = GaussianNLLLoss(full=True)
    ystd = torch.Tensor(ystd.reshape(-1,1))
    ymeans = torch.Tensor(ymeans.reshape(-1,1))
    ytrue = torch.Tensor(ytrue.reshape(-1,1))
    return loss(ymeans, ytrue, ystd**2)

def small_correction(alpha,M):
    return np.sqrt(-1/M + (M-3)/(M-1) * alpha**2)

def combined_pred(pred_1,pred_2,rescale_1,rescale_2,rescale_1_params,rescale_2_params):
    """passing a (N_samples,N_models) array of predictions
    """
    
    mean_1 = pred_1.mean(axis=1)
    mean_2 = pred_2.mean(axis=1)
    
    std_1 = pred_1.std(axis=1,ddof=1)
    std_2 = pred_2.std(axis=1,ddof=1)
    
    std_1 = rescale_1(std_1,*rescale_1_params)
    std_2 = rescale_2(std_2,*rescale_2_params)
    
    final_pred = (1/std_1**2 + 1/std_2**2)**-1 * (1/std_1**2 * mean_1 + 1/std_2**2 * mean_2)
    
    return final_pred

def dimensionless_coeff(LLworst,LLbest,LLactual):
        return max((LLworst-LLactual),0)/(LLworst-LLbest)*100
    
params_NN = {}

for a_specie in [1,6,7,8]:
    
    int_param = {"alpha_v1":None,"alpha_v2":None,"beta_v1":None,"beta_v2":None,"gamma_v1":None,"gamma_v2":None}
    
    pred_v1 = np.copy(result_dict["v1"]["pred_train"][a_specie])
    pred_v1_test = np.copy(result_dict["v1"]["pred_test"][a_specie])
    mean_v1 = np.copy(result_dict["v1"]["means_train"][a_specie])
    mean_v1_test = np.copy(result_dict["v1"]["means_test"][a_specie])
    
    pred_v2 = np.copy(result_dict["v2"]["pred_train"][a_specie])
    pred_v2_test = np.copy(result_dict["v2"]["pred_test"][a_specie])
    mean_v2 = np.copy(result_dict["v2"]["means_train"][a_specie])
    mean_v2_test = np.copy(result_dict["v2"]["means_test"][a_specie])
    
    truth = np.copy(result_dict["v1"]["train_true"][a_specie])
    truth_test = np.copy(result_dict["v1"]["test_true"][a_specie])


    one_hot_v1 = np.copy(result_dict["v1"]["one_hot"][a_specie])
    one_hot_v2 = np.copy(result_dict["v2"]["one_hot"][a_specie])
    
    #def loss(ymeans,ytrue,ystd):
    
    smaller5_v1 = np.sum(one_hot_v1,axis=1) < 11
    smaller5_v2 = np.sum(one_hot_v2,axis=1) < 11
    masked_commitee_pred_v1 = np.ma.masked_array(data=pred_v1,mask=one_hot_v1)
    masked_commitee_pred_v2 = np.ma.masked_array(data=pred_v2,mask=one_hot_v2)

    ystd_data_v1 = masked_commitee_pred_v1[smaller5_v1].std(ddof=1,axis=1)
    ymeans_data_v1 = masked_commitee_pred_v1[smaller5_v1].mean(axis=1)
    ytrue_data_v1 = np.copy(truth[smaller5_v1])

    ystd_data_v2 = masked_commitee_pred_v2[smaller5_v2].std(ddof=1,axis=1)
    ymeans_data_v2 = masked_commitee_pred_v2[smaller5_v2].mean(axis=1)
    ytrue_data_v2 = np.copy(truth[smaller5_v2])

    
    #get NLLs and params
    
    min_res_alpha_only_v1 = minimize(neglog_likelihood_rescale,args=(ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a),x0=np.array([1.]))
    alpha_only_v1 = min_res_alpha_only_v1["x"]
    NLL_alpha_only_v1 = min_res_alpha_only_v1["fun"]
    
    min_res_nonlin_v1 = minimize(neglog_likelihood_rescale,args=(ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a_g),x0=np.array([1.,1.]))
    alpha_nonlin_v1, gamma_nonlin_v1 = min_res_nonlin_v1["x"]
    NLL_nonlin_v1 = min_res_nonlin_v1["fun"]
    
    min_res_full_v1 = minimize(neglog_likelihood_rescale,args=(ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a_b_g),x0=np.array([1.,1.,1.]))
    alpha_v1, beta_v1, gamma_v1 = min_res_full_v1["x"]
    NLL_full_v1 = min_res_full_v1["fun"]
    
    min_res_alpha_only_v2 = minimize(neglog_likelihood_rescale,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2,rescale_a),x0=np.array([1.]))
    alpha_only_v2 = min_res_alpha_only_v2["x"]
    print(alpha_only_v2)
    NLL_alpha_only_v2 = min_res_alpha_only_v2["fun"]
    
    min_res_nonlin_v2 = minimize(neglog_likelihood_rescale,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2,rescale_a_g),x0=np.array([1.,1.]))
    alpha_nonlin_v2, gamma_nonlin_v2 = min_res_nonlin_v2["x"]
    NLL_nonlin_v2 = min_res_nonlin_v2["fun"]
    
    min_res_full_v2 = minimize(neglog_likelihood_rescale,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2,rescale_a_b_g),x0=np.array([1.,1.,1.]))
    alpha_v2, beta_v2, gamma_v2 = min_res_full_v2["x"]
    NLL_full_v2 = min_res_full_v2["fun"]
    
    neglog_likelihood_no_rescale_v1 = neglog_likelihood(pred_v1_test.std(axis=1,ddof=1),mean_v1_test,truth_test)
    neglog_likelihood_no_rescale_v2 = neglog_likelihood(pred_v2_test.std(axis=1,ddof=1),mean_v2_test,truth_test)
    #test set RMSEs
    
    RMSE_v1 = mean_squared_error(truth_test,mean_v1_test,squared=False)
    RMSE_v2 = mean_squared_error(truth_test,mean_v2_test,squared=False)
    
    errors_v1 = np.abs(truth_test-mean_v1_test)
    errors_v2 = np.abs(truth_test-mean_v2_test)
    
    NLL_test_v1_a = neglog_likelihood_rescale([small_correction(alpha_only_v1,16)],pred_v1_test.std(axis=1,ddof=1),mean_v1_test,truth_test,rescale_a)
    NLL_test_v1_a_g = neglog_likelihood_rescale([alpha_nonlin_v1, gamma_nonlin_v1],pred_v1_test.std(axis=1,ddof=1),mean_v1_test,truth_test,rescale_a_g)
    NLL_test_v1_a_b_g = neglog_likelihood_rescale([alpha_v1, beta_v1, gamma_v1],pred_v1_test.std(axis=1,ddof=1),mean_v1_test,truth_test,rescale_a_b_g)
    NLL_test_v2_a = neglog_likelihood_rescale([small_correction(alpha_only_v2,16)],pred_v2_test.std(axis=1,ddof=1),mean_v2_test,truth_test,rescale_a)
    NLL_test_v2_a_g = neglog_likelihood_rescale([alpha_nonlin_v2, gamma_nonlin_v2],pred_v2_test.std(axis=1,ddof=1),mean_v2_test,truth_test,rescale_a_g)
    NLL_test_v2_a_b_g = neglog_likelihood_rescale([alpha_v2, beta_v2, gamma_v2],pred_v2_test.std(axis=1,ddof=1),mean_v2_test,truth_test,rescale_a_b_g)
    
    NLL_worst_v1 = neglog_likelihood(RMSE_v1*np.ones(mean_v1_test.shape),mean_v1_test,truth_test)
    NLL_best_v1 = neglog_likelihood(errors_v1,mean_v1_test,truth_test)
     
    NLL_worst_v2 = neglog_likelihood(RMSE_v2*np.ones(mean_v2_test.shape),mean_v2_test,truth_test)
    NLL_best_v2 = neglog_likelihood(errors_v2,mean_v2_test,truth_test)
    
    print("NLL v1 no rescale: {:.2f}".format(neglog_likelihood_no_rescale_v1))
    print("NLL v1 alpha rescale: {:.2f}".format(NLL_alpha_only_v1))
    print("NLL v1 a,b,g rescale: {:.2f}".format(NLL_full_v1))
    print("NLL v1 worst: {:.2f}".format(NLL_worst_v1))
    print("NLL v1 best: {:.2f}".format(NLL_best_v1))
    
    print("NLL v2 no rescale: {:.2f}".format(neglog_likelihood_no_rescale_v2))
    print("NLL v2 alpha rescale: {:.2f}".format(NLL_test_v2_a))
    print("NLL v2 a,b,g, rescale: {:.2f}".format(NLL_test_v2_a_b_g))
    print("NLL v2 worst: {:.2f}".format(NLL_worst_v2))
    print("NLL v2 best: {:.2f}".format(NLL_best_v2))

    
    print("species: {}, v1 before rescale : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v1,NLL_best_v1,neglog_likelihood_no_rescale_v1)))
    print("species: {}, v1 rescale a : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL_test_v1_a)))
    print("species: {}, v1 rescale a,g : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL_test_v1_a_g)))
    print("species: {}, v1 rescale a,b,g : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL_test_v1_a_b_g)))
    
    print("species: {}, v2 before rescale : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v2,NLL_best_v2,neglog_likelihood_no_rescale_v2)))
    print("species: {}, v2 rescale a : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL_test_v2_a)))
    print("species: {}, v2 rescale a,g : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL_test_v2_a_g)))
    print("species: {}, v2 rescale a,b,g : {:.2f}".format(a_specie,dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL_test_v2_a_b_g)))
    
    
    int_param["alpha_v1"] = alpha_v1
    int_param["alpha_v2"] = alpha_v2
    int_param["beta_v1"] = beta_v1
    int_param["beta_v2"] = beta_v2
    int_param["gamma_v1"] = gamma_v1
    int_param["gamma_v2"] = gamma_v2
    
    params_NN[a_specie] = int_param
    
    #------------lazy testing------------
    
    if testing is True:
        
        #print("Testing")
        
        for NLL in [NLL_alpha_only_v1,NLL_nonlin_v1,NLL_full_v1]:
            assert dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL) <= 100
            assert dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL) >= 0
            assert dimensionless_coeff(NLL_worst_v1,NLL_best_v1,NLL_best_v1) == 100
        
        for NLL in [NLL_alpha_only_v2,NLL_nonlin_v2,NLL_full_v2]:
            assert dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL) <= 100
            assert dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL) >= 0
            assert dimensionless_coeff(NLL_worst_v2,NLL_best_v2,NLL_best_v2) == 100
        
        assert NLL_worst_v1 != NLL_worst_v2
        assert NLL_best_v1 != NLL_best_v2
        assert NLL_best_v1 < NLL_worst_v1
        assert NLL_best_v2 < NLL_worst_v2
        assert NLL_best_v1 < NLL_alpha_only_v1
        assert NLL_best_v2 < NLL_alpha_only_v2
        assert NLL_best_v1 < NLL_full_v1
        assert NLL_best_v2 < NLL_full_v2
        assert NLL_best_v1 < NLL_nonlin_v1
        assert NLL_best_v2 < NLL_nonlin_v2
        
        assert alpha_nonlin_v1 != alpha_nonlin_v2
        assert gamma_nonlin_v1 != gamma_nonlin_v2
        assert alpha_only_v1 != alpha_only_v2
        assert alpha_v1 != alpha_v2
        assert beta_v1 != beta_v2
        assert gamma_v1 != gamma_v2
        
        assert NLL_alpha_only_v1 != NLL_alpha_only_v2
        assert NLL_full_v1 != NLL_full_v2
        assert NLL_nonlin_v1 != NLL_nonlin_v2
        
        assert np.allclose(neglog_likelihood_rescale(alpha_only_v1,ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a),NLL_alpha_only_v1)
        assert np.allclose(neglog_likelihood_rescale([alpha_nonlin_v1, gamma_nonlin_v1],ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a_g),NLL_alpha_only_v1) is False
        assert np.allclose(neglog_likelihood_rescale([alpha_nonlin_v1, gamma_nonlin_v1],ystd_data_v1,ymeans_data_v1,ytrue_data_v1,rescale_a_g),NLL_nonlin_v1)
        
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a,rescale_a,[alpha_only_v1],[alpha_only_v1])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_g,rescale_a_g,[alpha_nonlin_v1,gamma_nonlin_v1],[alpha_only_v1,gamma_nonlin_v1])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_b_g,rescale_a_b_g,[alpha_v1,beta_v1,gamma_v1],[alpha_v1,beta_v1,gamma_v1])
        assert np.allclose(combined_v1,mean_v1_test)
        
        
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a,rescale_a,[alpha_only_v1],[alpha_only_v1])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a,rescale_a,[alpha_only_v1],[1.])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a,rescale_a_b_g,[alpha_only_v1],[1.,1.,1.])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_b_g,rescale_a_b_g,[1.,1.,1.],[1.,1.,1.])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_b_g,rescale_a_b_g,[1.,0.1,1.],[0.1,1.,0.1])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_b_g,rescale_a_b_g,[1.,0.1,1.],[0.1,1.,0.1])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_g,rescale_a_g,[1.,1.],[1.,1.])
        assert np.allclose(combined_v1,mean_v1_test)
        combined_v1 = combined_pred(pred_v1_test,pred_v1_test,rescale_a_g,rescale_a_g,[1.,0.1],[0.1,1.])
        assert np.allclose(combined_v1,mean_v1_test)        
        
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a,rescale_a,[alpha_only_v2],[alpha_only_v2])
        assert np.allclose(combined_v2,mean_v2_test)
        
        #ensure that combinations differ
        assert np.allclose(combined_v1,combined_v2) is False
        
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_g,rescale_a_g,[alpha_nonlin_v2,gamma_nonlin_v2],[alpha_nonlin_v2,gamma_nonlin_v2])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_b_g,rescale_a_b_g,[alpha_v2,beta_v2,gamma_v2],[alpha_v2,beta_v2,gamma_v2])
        assert np.allclose(combined_v2,mean_v2_test)
        
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a,rescale_a,[alpha_only_v2],[alpha_only_v2])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a,rescale_a,[alpha_only_v2],[1.])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a,rescale_a_b_g,[alpha_only_v2],[1.,1.,1.])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_b_g,rescale_a_b_g,[1.,1.,1.],[1.,1.,1.])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_b_g,rescale_a_b_g,[1.,0.1,1.],[0.1,1.,0.1])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_b_g,rescale_a_b_g,[1.,0.1,1.],[0.1,1.,0.1])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_g,rescale_a_g,[1.,1.],[1.,1.])
        assert np.allclose(combined_v2,mean_v2_test)
        combined_v2 = combined_pred(pred_v2_test,pred_v2_test,rescale_a_g,rescale_a_g,[1.,0.1],[0.1,1.])
        assert np.allclose(combined_v2,mean_v2_test)
        
        combined_v1_v2 = combined_pred(pred_v1_test,pred_v2_test,rescale_a_g,rescale_a_g,[alpha_nonlin_v1,gamma_nonlin_v1],[alpha_nonlin_v2,gamma_nonlin_v2])
        combined_v2_v1 = combined_pred(pred_v2_test,pred_v1_test,rescale_a_g,rescale_a_g,[alpha_nonlin_v2,gamma_nonlin_v2],[alpha_nonlin_v1,gamma_nonlin_v1])
        #print(np.abs(combined_v1_v2-combined_v2_v1))
        assert np.allclose(combined_v1_v2,combined_v2_v1)
        
        
        combined_v1_v2 = combined_pred(pred_v1_test,pred_v2_test,rescale_a_b_g,rescale_a_g,[alpha_v1,beta_v1,gamma_v1],[alpha_nonlin_v2,gamma_nonlin_v2])
        combined_v2_v1 = combined_pred(pred_v2_test,pred_v1_test,rescale_a_g,rescale_a_b_g,[alpha_nonlin_v2,gamma_nonlin_v2],[alpha_v1,beta_v1,gamma_v1])
        #print(np.abs(combined_v1_v2-combined_v2_v1))
        assert np.allclose(combined_v1_v2,combined_v2_v1)
        
    
        if a_specie == 1:
            pass
            print(small_correction(alpha_only_v2,16))
            print(RMSE_v2)
            assert np.allclose(small_correction(alpha_only_v2,16),5.02753776,atol=1e-04)
            assert(np.allclose(RMSE_v2,0.4777233772507114)) 
    
    
       
#with open("rescaling_params_NN.json","w") as fg:
#    json.dump(params_NN,fg)
    

[5.40711454]
NLL v1 no rescale: 933.20
NLL v1 alpha rescale: 1.64
NLL v1 a,b,g rescale: 1.52
NLL v1 worst: 1.61
NLL v1 best: 0.91
NLL v2 no rescale: 13.31
NLL v2 alpha rescale: 0.69
NLL v2 a,b,g, rescale: 0.65
NLL v2 worst: 0.68
NLL v2 best: -0.04
species: 1, v1 before rescale : 0.00
species: 1, v1 rescale a : 0.00
species: 1, v1 rescale a,g : 2.52
species: 1, v1 rescale a,b,g : 2.41
species: 1, v2 before rescale : 0.00
species: 1, v2 rescale a : 0.00
species: 1, v2 rescale a,g : 4.68
species: 1, v2 rescale a,b,g : 4.84
[5.02753776]
0.4777233772507114
[4.41584666]
NLL v1 no rescale: 454.34
NLL v1 alpha rescale: 3.86
NLL v1 a,b,g rescale: 3.72
NLL v1 worst: 3.65
NLL v1 best: 2.84
NLL v2 no rescale: 9.66
NLL v2 alpha rescale: 2.85
NLL v2 a,b,g, rescale: 2.82
NLL v2 worst: 2.91
NLL v2 best: 2.09
species: 6, v1 before rescale : 0.00
species: 6, v1 rescale a : 11.24
species: 6, v1 rescale a,g : 9.20
species: 6, v1 rescale a,b,g : 9.20
species: 6, v2 before rescale : 0.00
species: 6, v2 resc

In [10]:
alpha_only_v2

array([3.91948738])

In [10]:
species: 1, comb a : 0.00
species: 1, comb a,g : 12.77
species: 1, comb a,b,g : 12.82
species: 1, comb a : 0.00
species: 1, comb a,g : 0.00
species: 1, comb a,b,g : 0.00
Testing
[5.02753776]
0.4777233772507114
species: 6, comb a : 0.00
species: 6, comb a,g : 0.00
species: 6, comb a,b,g : 0.00
species: 6, comb a : 0.00
species: 6, comb a,g : 0.00
species: 6, comb a,b,g : 0.00
Testing
species: 7, comb a : 0.00
species: 7, comb a,g : 2.21
species: 7, comb a,b,g : 2.22
species: 7, comb a : 16.80
species: 7, comb a,g : 20.31
species: 7, comb a,b,g : 21.79
Testing
species: 8, comb a : 5.66
species: 8, comb a,g : 15.46
species: 8, comb a,b,g : 15.48
species: 8, comb a : 0.00
species: 8, comb a,g : 0.00
species: 8, comb a,b,g : 0.00

SyntaxError: invalid syntax (946170468.py, line 1)

In [21]:
assert 1 == 2

AssertionError: 

In [64]:
torch LL: 13.299803823304574
my LL: 13.299802780151367
[5.40711451]
1.2329739441615915 -1.216211102153372
2.312352664107572 0.3817045349494603 -0.18021740322433935
[5.02753773]
torch LL: 10.855299993159452
my LL: 10.85529899597168
[4.41584696]
4.339935318998906 -1.0345406553572736
2.5363892609863417 3.1813801660338137 -0.034760850348700904
[4.10332107]
torch LL: 11.222066260903093
my LL: 11.222066879272461
[4.31009723]
5.417401460339975 -0.5185490882038019
3.1058851749568053 5.136410301275955 0.14370665262811044
[4.00468639]
torch LL: 10.173645105018178
my LL: 10.17364501953125
[3.91948771]
7.1955332660982325 -0.8411707875486565
1.8284555683297692 11.748788676836176 0.35652090500182354
[3.64027005]


SyntaxError: invalid syntax (3245100638.py, line 1)

In [60]:
b = np.array([1,2,3])
print(*b)

1 2 3


In [ ]:
torch LL: 13.299803823304574
my LL: 13.299802780151367
[5.40711451]
1.2329739441615915 -1.216211102153372
2.312513144766872 0.38171130785928264 -0.18011772488824487
[5.02753773]
torch LL: 10.855299993159452
my LL: 10.85529899597168
[4.41584696]
4.339935318998906 -1.0345406553572736
4.339935505151607 1.2237235956918827e-05 -1.0345404704806083
[4.10332107]
torch LL: 11.222066260903093
my LL: 11.222066879272461
[4.31009723]
5.417401460339975 -0.5185490882038019
5.417401460339975 0.0 -0.5185490882038019
[4.00468639]
torch LL: 10.173645105018178
my LL: 10.17364501953125
[3.91948771]
7.1955332660982325 -0.8411707875486565
7.195551216400041 0.0 -0.8411736291447887
[3.64027005]

In [49]:
minimize(neglog_likelihood_a_b_c,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.,0.,0.]))

      fun: 4.303959087282078
 hess_inv: array([[235.31736518,   0.        , -37.02211963],
       [  0.        ,   1.        ,   0.        ],
       [-37.02211963,   0.        ,   6.61289986]])
      jac: array([3.57627869e-07, 0.00000000e+00, 2.08616257e-06])
  message: 'Optimization terminated successfully.'
     nfev: 72
      nit: 17
     njev: 18
   status: 0
  success: True
        x: array([ 7.19555122,  0.        , -0.84117363])

## Graveyard

In [66]:
    """
    def neglog_likelihood_a(alpha,ystd,ymeans,ytrue,nll=neglog_likelihood):
        ystd = alpha * ystd 
        return nll(ystd,ymeans,ytrue)

    def neglog_likelihood_a_g(x,ystd,ymeans,ytrue,nll=neglog_likelihood):
        alpha = x[0]
        gamma = x[1]
        ystd = np.sqrt(alpha**2 * ystd**(gamma+2))
        return nll(ystd,ymeans,ytrue)    
    
    def neglog_likelihood_a_b_g(x,ystd,ymeans,ytrue,nll=neglog_likelihood):
        alpha = x[0]
        beta = x[1]
        gamma = x[2]
        ystd = np.sqrt(alpha**2 * ystd**(gamma+2) + beta**2)
        return nll(ystd,ymeans,ytrue)
        

    alpha_only = minimize(neglog_likelihood_a,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.]))["x"]
    alpha_nonlin, gamma_nonlin = minimize(neglog_likelihood_a_g,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.,0.]))["x"]
    alpha, beta, gamma = minimize(neglog_likelihood_a_b_g,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.,0.,0.]))["x"]
    
    """
        #print("torch LL: {}".format(neglog_likelihood(ystd_data_v2,ymeans_data_v2,ytrue_data_v2)))
    #print("my LL: {}".format(neglog_likelihood_torch(ystd_data_v2,ymeans_data_v2,ytrue_data_v2)))

'\ndef neglog_likelihood_a(alpha,ystd,ymeans,ytrue,nll=neglog_likelihood):\n    ystd = alpha * ystd \n    return nll(ystd,ymeans,ytrue)\n\ndef neglog_likelihood_a_g(x,ystd,ymeans,ytrue,nll=neglog_likelihood):\n    alpha = x[0]\n    gamma = x[1]\n    ystd = np.sqrt(alpha**2 * ystd**(gamma+2))\n    return nll(ystd,ymeans,ytrue)    \n\ndef neglog_likelihood_a_b_g(x,ystd,ymeans,ytrue,nll=neglog_likelihood):\n    alpha = x[0]\n    beta = x[1]\n    gamma = x[2]\n    ystd = np.sqrt(alpha**2 * ystd**(gamma+2) + beta**2)\n    return nll(ystd,ymeans,ytrue)\n    \n\nalpha_only = minimize(neglog_likelihood_a,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.]))["x"]\nalpha_nonlin, gamma_nonlin = minimize(neglog_likelihood_a_g,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.,0.]))["x"]\nalpha, beta, gamma = minimize(neglog_likelihood_a_b_g,args=(ystd_data_v2,ymeans_data_v2,ytrue_data_v2),x0=np.array([1.,0.,0.]))["x"]\n\n'

In [ ]:
    """
    example:
    torch LL: 0.6961718892174772
    my LL: 0.6961718892174772
    """